## MP-3 Project - Analysis of ESC Semifinals running order and results

In [ ]:
import pandas as pd, numpy as np, seaborn as sbn, matplotlib.pyplot as plt

from Modules import utils as utl
from Modules import data_exploration as de
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import sklearn.metrics as sm
import pickle

In [ ]:
csv_file_path = 'Data/semifinalists_cleaned.csv'
eurovision_semi_df = utl.load_csv(csv_file_path, skiprows=0, encoding='windows-1252')

In [ ]:
eurovision_semi_df

In [ ]:
eurovision_semi_df.shape

In [ ]:
# visualise the features and the response using scatterplots
sbn.pairplot(eurovision_semi_df, x_vars=['semi_draw_position'], y_vars='semi_place', height=5, aspect=0.8)
plt.show()

In [ ]:
# histograms
hist = eurovision_semi_df.hist()

### Trying to figure out why there is a gap between 7.5 and 10 in semi_draw_position and semi_place

In [ ]:
eurovision_semi_df['semi_draw_position'].hist()

In [ ]:
not_semi_places = ['semi_final', 'country', 'style', 'semi_televote_points', 'semi_jury_points', 'semi_total_points']

In [ ]:
semi_places = eurovision_semi_df.drop(columns=not_semi_places)

In [ ]:
semi_places.info()

In [ ]:
# Count the number of occurrences of each place in 'semi_place'
place_counts = eurovision_semi_df['semi_place'].value_counts().sort_index()

# Display the counts
print(place_counts)


In [ ]:
# Count the number of occurrences of each place in 'semi_place'
place_counts = eurovision_semi_df['semi_draw_position'].value_counts().sort_index()

# Display the counts
print(place_counts)
# the reason for the 2 missing entries at semi_draw_position 1 - is because those 2 entries are missing semi_place data, so they are quite difficult to work around.

In [ ]:
# Define custom bin edges to combat a skewed distribution in the intervals - as the positions will always be integers and never floating points - so the intervals are misleading
bin_edges = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

# Plot histogram with custom bin edges
plt.hist(eurovision_semi_df['semi_draw_position'], bins=bin_edges, edgecolor='black')
plt.title('Distribution of Semi-Positions')
plt.xlabel('Semi-Positions')
plt.ylabel('Count')
plt.show()


In [ ]:
# Define custom bin edges to combat a skewed distribution in the intervals - as the placings will always be integers and never floating points - so the intervals are misleading
bin_edges = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

# Plot histogram with custom bin edges
plt.hist(eurovision_semi_df['semi_place'], bins=bin_edges, edgecolor='black')
plt.title('Distribution of Semi-Places')
plt.xlabel('Semi-Places')
plt.ylabel('Count')
plt.show()
# The two missing entries actually account for the small gap at 10 and 11 - the entries are Montenegro 2009 (11th) and Moldova 2010 (10th)

#### I have made more accurate bin distribution in the two histograms, which shows that there is a consistant pattern, with only 2 entries missing due to missing data in the dataset. So even though the initial histograms looked suspicious they were quite misleading.

### Back to exploration of data

In [ ]:
eurovision_semi_df.isnull().sum()

In [ ]:
eurovision_semi_df.describe()

In [ ]:
de.vs.lmplot(eurovision_semi_df, 'semi_draw_position', 'semi_place')
# the smaller the semi_place - the better

In [ ]:
de.vs.boxplot(eurovision_semi_df, 'semi_draw_position', 'semi_place')

In [ ]:
# reformat the Eurovision semifinal list to not include strings
eurovision_semi_num = ['year', 'semi_final', 'semi_draw_position', 'semi_televote_points', 'semi_jury_points', 'semi_total_points', 'semi_place']
eurovision_semi_df = eurovision_semi_df[eurovision_semi_num]

In [ ]:
de.vs.correlation_heatmap(eurovision_semi_df)

### Separating the non numeral data from the data we want to train and test

In [ ]:
# create a Python list of feature names
feature_cols = ['semi_final', 'semi_draw_position']

# use the list to select a subset of the original DataFrame
X = eurovision_semi_df[feature_cols]

# print the first 5 rows
X.head()

In [ ]:
# select a Series from the DataFrame for y
y = eurovision_semi_df['semi_place']

# print the first 5 values
y.head()

In [ ]:
# check the type and shape of X
print(type(X))
print(X.shape)

In [ ]:
# check the type and shape of y
print(type(y))
print(y.shape)

In [ ]:
import sys
import os

# Use the current working directory instead of __file__
current_directory = os.getcwd()

# Add the path to the 'Modules' folder (adjust the path to reach the Modules folder)
sys.path.append(os.path.join(current_directory, 'Modules'))

# Now import the evalute function
import machine_learning
from machine_learning import *

### Training and testing

In [ ]:
# training the data
X_train, X_test, y_train, y_test = machine_learning.prepare_data(eurovision_semi_df, 'semi_place')

In [ ]:
# using Pipeline to combat NaN values in the points columns - and replacing them with a "mean" aggregated result instead.
model = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('regressor', LinearRegression())
])

In [ ]:
model.fit(X_train, y_train)

In [ ]:
reg = model.named_steps['regressor']

In [ ]:
print("Intercept:", reg.intercept_)
print("Coefficients:", reg.coef_)

feature_names = X_train.columns
coef_table = list(zip(feature_names, reg.coef_))
for name, coef in coef_table:
    print(f"{name:30} {coef:>10.4f}")


####
Intercept: -536.99
 The intercept is the predicted semifinal place when all features are 0, which doesn't have much practical interpretation in this case. But in case it was - a negative value that big is reason for concern that the values aren't coherent with each other.

 Coefficients:
1. 'Year' (0.2738):
       A one-year increase is associated with a slight increase (0.27) in the predicted semifinal place. 
        This suggests that over the years, the rankings have slightly moved upwards on average - aka they have gotten worse.
2. 'Semi Final' (0.1695): 
    Moving from one semifinal to another is associated with a small increase (0.17) in the predicted place (so semi 2 is worse than semi 1), 
    reflecting possible differences in competition between the two semifinals. Perhaps because when the number of semifinalists are odd, one of the semifinals will have one more entry than the other - making it less likely to qualify from that semifinal.
3. 'Semi Draw Position' (-0.0332): 
    A later performance position (higher number) is associated with a slight improvement (lower `semi_place`) in ranking. 
    This supports the idea that later performances might be more memorable or have an advantage, potentially due to recency bias.
4. 'Semi Televote Points' (-0.0048): 
    As televote points increase, the predicted semifinal place improves (i.e., a better placement). 
    This is intuitive since more televote points would likely lead to a better semifinal result.
5. 'Semi Jury Points' (0.0093): 
    A higher number of jury points is associated with a worse semifinal place (higher `semi_place`), which is unexpected and suggests that higher jury points alone don't necessarily guarantee a better place in the semifinal. Perhaps the televote disagrees more strongly with jury favorites than juries do with televote favorites, leading to jury favorites performing slightly worse overall in the total result? As we also have to keep in mind the total result have in most of these recorded years (2009-2023) been 50%/50% jury/televote based - with the exception of 2023 and up - where the juries were removed from semifinals entirely, but still remaining at the same capacity in the final.
6. 'Semi Total Points' (-0.0597): 
    An increase in total points (combination of jury and televote) is associated with a better semifinal position 
    (lower `semi_place`), which is consistent with the idea that more points lead to a higher placement.

A negative coefficient generally represents a negative correlation with semi_place - which actually is good - because that means the value in question "negatively" impacts the place result (the number becomes lower) - as the lower the place number is - the better it is - as 1 is the best result here. So fx a high number of point should give you a low place number - like 3rd place, and vice versa a low number of points would give you a "high" placing like 15th - which would make it less likely to qualify.


In [ ]:
correlation_matrix = eurovision_semi_df[['semi_jury_points', 'semi_televote_points', 'semi_total_points']].corr()
print(correlation_matrix)


##### Jury points seems to align slightly more with the final result than the televote points

In [ ]:
machine_learning.evaluate_model(model, X_test, y_test)

In [ ]:
#machine_learning.save_model(model, 'semifinalpredictions')

In [ ]:
load_model('semifinalpredictions')

In [ ]:

from sklearn import metrics

y_predicted = model.predict(X_test)

print("Mean Absolute Error (MAE) = ", metrics.mean_absolute_error(y_test, y_predicted))

print("Mean Squared Error (MSE) = ", metrics.mean_squared_error(y_test, y_predicted))


# calculate RMSE using scikit-learn
print("Root Mean Squared Error (RMSE) = ", np.sqrt(metrics.mean_squared_error(y_test, y_predicted)))


# R-squared
r2_score = metrics.r2_score(y_test, y_predicted)
print ("R2 (R-squared) score = ", r2_score)
eV = round(sm.explained_variance_score(y_test, y_predicted), 4)
print('Explained variance score = ',eV )

# Visualise the regression results
plt.title('Multiple Linear Regression')
plt.scatter(y_test, y_predicted, color='magenta')
plt.show()




##### 
- MAE = the predicted semifinal places are off by approximately 2.2 places - so a 10th place here could be between 7.8 and 12.2 in the actual result - which is pretty close.
- MSE = A value of 7.04 indicates that, on average, the squared difference between predicted and actual places is 7.04, which in particular highlights the scale of this regression's errors due to the usage of squaring.
- RMSE = A value of 2.65 means that, on average, the predicted semifinal place is off by around 2.65 places - which is also still reasonable.
- Both R2 and the variance score are 0.68 - which means the model has covered 68% of the total variance in the semifinal results.

In [ ]:
residuals = y_test - y_predicted
plt.scatter(y_predicted, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title("What is the deviation between predicted vs actual results? The line is the actual result")
plt.xlabel("Predicted")
plt.ylabel("Residuals")
plt.show()


In [ ]:
plt.scatter(y_test, y_predicted, color='magenta')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')  # Line of perfect prediction
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs Predicted")
plt.show()


#### Since Eurovision results have a tendency to be quite unpredictable, we don't expect these predictions to be entirely accurate. But an variance score of 68% is pretty alright for a song contest with a small number of possible positions - in theory up until 26 - but ususally less than 20. That means of course that there is 32% of the result variance in the semifinals we don't account for. This can generally just stem from the upredictability of Eurovision results - like country-specific tendencies (diaspora voting, political voting, bloc-voting etc.), or just disrepancies between the juries and televotes.